# WELCOME TO DATA SCIENCE CAPSTONE PROJECT - WEEK 3
## For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
### Part 1:
First, we import the necessary libraries

In [167]:
import requests
import lxml.html as lh
import pandas as pd

Create a handle, page, to handle the contents of the website. Then, Store the contents of the website under doc and finally, Parse data that are stored between tr of HTML

In [168]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Create an empty list and for each row, store each first element (header) and an empty list

In [169]:
col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



Then, we are going to go for any row and assign the value to the above list.
If row is not of size 3, the //tr data is not from our table.

In [170]:
for j in range(1,len(tr_elements)):

    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

Create a Dictionary and then, assign the dictionary to a dataframe

In [171]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Filter the Dataframe when Borough is not equal to "Not Assigned", then, replace the end of lines for each row and change the column name by Neighbourhood

In [172]:
df_torontoPC = df[df['Borough']!='Not assigned']
df_torontoPC.is_copy = None
df_torontoPC['Neighbourhood\n'] = df_torontoPC['Neighbourhood\n'].str[:-1]
df_torontoPC.rename(columns={'Neighbourhood\n':'Neighbourhood'}, inplace=True)
df_torontoPC['Neighbourhood'].replace('Not assigned',df_torontoPC['Borough'], inplace=True)
df_torontoPC = df_torontoPC.sort_values('Neighbourhood').groupby('Postcode').agg({'Borough':'first','Neighbourhood':', '.join})
df_torontoPC.reset_index(inplace=True)
df_torontoPC.head()
#postcode=['M5G','M2H','M4B','M1J']
#df_torontoPC[df_torontoPC.Postcode.isin(postcode)]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [173]:
df_torontoPC.shape

(103, 3)

### Part 2:

First, we need to install de geocoder package

In [174]:
#conda update -n base -c defaults conda
!conda install -c conda-forge geocoder -y

SyntaxError: invalid syntax (<ipython-input-174-1c63f85a465d>, line 1)

Then, we create a function to request the coordinates by postal code:

The function receives as parameter the postal code and then, return the coordinates

In [175]:
import geocoder # import geocoder
postal_code='M5G'
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print("Latitud: ", latitude)
print("Longitud: ", longitude)

Latitud:  43.65609081300005
Longitud:  -79.38492999999994


We need to do that for all of the Postal Codes to our dataframe, so we create a loop in for to that:

In [176]:
postalcodes=df_torontoPC.Postcode.unique()

In [177]:
import geocoder
latitudes = []
longitudes = []
codigosPostales = []
for i in range(len(postalcodes)):
    codigoPostal = postalcodes[i]
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(codigoPostal))
    #print("Latitud: ", g.latlng[0], " Longitud: ", g.latlng[1], " del codigo Postal: ", codigoPostal)
    latitudes.append(g.latlng[0])
    longitudes.append(g.latlng[1])
    codigosPostales.append(codigoPostal)

Then, we convert the lists in a DataFrame to merge with the first dataframe created

In [178]:
list_of_tuples = list(zip(latitudes, longitudes, codigosPostales)) 
coordenadasCP_df = pd.DataFrame(list_of_tuples, columns = ['Latitud','Longitud','CodigoPostal'])
coordenadasCP_df.head()

,Latitud,Longitud,CodigoPostal
0,43.811525,-79.195517,M1B
1,43.785665,-79.158725,M1C
2,43.765815,-79.175193,M1E
3,43.768369,-79.217590,M1G
4,43.769688,-79.239440,M1H


For last, we merge both dataframes and drop the column duplicated

In [179]:
mergeDF = pd.merge(left=df_torontoPC, right=coordenadasCP_df, how='left', left_on='Postcode', right_on='CodigoPostal')
mergeDF.drop(['CodigoPostal'], axis=1, inplace=True)
mergeDF.head()

,Postcode,Borough,Neighbourhood,Latitud,Longitud
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Part 3: